# https://github.com/athina-ai/rag-cookbooks を元に8種類のRAGと5種類のAgenticRAGについて学ぶ 2/13

# Hybrid RAG
ハイブリッドRAGとは、ベクトル類似性検索と全文検索やBM25のような従来の検索手法を組み合わせた高度な検索手法のことである。このアプローチは、意味理解のためのベクトル類似性と、正確なキーワードやテキストベースのマッチングのための伝統的な手法、両方の長所を活用することで、より包括的で柔軟な情報検索を可能にする。

論文: [paper1](https://arxiv.org/pdf/2408.05141) and [paper2](https://arxiv.org/pdf/2408.04948)

# Environment building

In [1]:
!pip install -qq -U  langchain-openai langchain-community chromadb rank_bm25

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Indexing

In [3]:
import os
from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
os.environ['ATHINA_API_KEY'] = userdata.get('ATHINA_API_KEY')
# os.environ['PINECONE_API_KEY'] = userdata.get('PINECONE_API_KEY')

In [4]:
# load embedding model
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

In [5]:
# load data
from langchain.document_loaders import CSVLoader
loader = CSVLoader("/content/drive/MyDrive/Colab Notebooks/rag-cookbooks-copy/data/context.csv")
documents = loader.load()

In [6]:
# split documents
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
documents = text_splitter.split_documents(documents)

In [7]:
# create vectorstore
from langchain.vectorstores import Chroma
vectorstore = Chroma.from_documents(documents, embeddings)

# Retriever

In [8]:
# create retriever
retriever = vectorstore.as_retriever()

# Keyword Retriever

In [9]:
# create keyword retriever
from langchain.retrievers import BM25Retriever
keyword_retriever = BM25Retriever.from_documents(documents)
keyword_retriever.k = 3

In [10]:
# test keyword retriever
keyword_retriever.get_relevant_documents("what bacteria grow on macconkey agar")

<ipython-input-10-5b9946a99a06>:2: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  keyword_retriever.get_relevant_documents("what bacteria grow on macconkey agar")


[Document(metadata={'source': '/content/drive/MyDrive/Colab Notebooks/rag-cookbooks-copy/data/context.csv', 'row': 6}, page_content='predominantly made from the lactose sugar in the agar.\\n\\n\\n== Variant ==\\nA variant, sorbitol-MacConkey agar, (with the addition of additional selective agents) can assist in the isolation and differentiation of enterohemorrhagic E. coli serotype E. coli O157:H7, by the presence of colorless circular colonies that are non-sorbitol fermenting.\\n\\n\\n== See also ==\\nR2a agar\\nMRS agar (culture medium designed to grow Gram-positive bacteria and differentiate them for lactose fermentation).\\n\\n\\n=='),
 Document(metadata={'source': '/content/drive/MyDrive/Colab Notebooks/rag-cookbooks-copy/data/context.csv', 'row': 37}, page_content='zoonotic disease since around 1910, but in the 1930s knowledge was gained that the bacteria lost their virulent power when repeatedly spread on agar media. This explained the difficulties to reproduce results from diff

# Ensamble Retriever

In [11]:
# create ensemble retriever
from langchain.retrievers import EnsembleRetriever
ensemble_retriever = EnsembleRetriever(retrievers=[retriever, keyword_retriever], weights=[0.5, 0.5])

In [12]:
# test emsemble retriever
ensemble_retriever.get_relevant_documents("what bacteria grow on macconkey agar")

[Document(metadata={'row': 6, 'source': '/content/drive/MyDrive/Colab Notebooks/rag-cookbooks-copy/data/context.csv'}, page_content="context: ['MacConkey agar is a selective and differential culture medium for bacteria. It is designed to selectively isolate Gram-negative and enteric (normally found in the intestinal tract) bacteria and differentiate them based on lactose fermentation. Lactose fermenters turn red or pink on MacConkey agar, and nonfermenters do not change color. The media inhibits growth of Gram-positive organisms with crystal violet and bile salts, allowing for the selection and isolation of gram-negative"),
 Document(metadata={'row': 6, 'source': '/content/drive/MyDrive/Colab Notebooks/rag-cookbooks-copy/data/context.csv'}, page_content='predominantly made from the lactose sugar in the agar.\\n\\n\\n== Variant ==\\nA variant, sorbitol-MacConkey agar, (with the addition of additional selective agents) can assist in the isolation and differentiation of enterohemorrhagic 

# RAG Chain

In [13]:
# load llm
from langchain_openai import ChatOpenAI
llm = ChatOpenAI()

In [14]:
# create document chain
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

template = """"
You are a helpful assistant that answers questions based on the following context.
If you don't find the answer in the context, just say that you don't know.
Context: {context}

Question: {input}

Answer:

"""
prompt = ChatPromptTemplate.from_template(template)

# setup RAG pipeline
rag_chain = (
    {"context": retriever, "input": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)


In [15]:
# response
response = rag_chain.invoke("what bacteria grow on macconkey agar")
response

'MacConkey agar is designed to selectively isolate Gram-negative and enteric (normally found in the intestinal tract) bacteria. It can differentiate bacteria based on lactose fermentation. Lactose fermenters turn red or pink on MacConkey agar, while nonfermenters do not change color. The media inhibits the growth of Gram-positive organisms, allowing for the selection and isolation of gram-negative bacteria.'

# Preparing Data for Evaluation

In [16]:
# create dataset
question = ["what bacteria grow on macconkey agar", "who wrote a rose is a rose is a rose"]
response = []
contexts = []

# inference
for query in question:
    print(query)
    response.append(rag_chain.invoke(query))
    contexts.append([docs.page_content for docs in ensemble_retriever.get_relevant_documents(query)])

# retriever.get_relevant_documents(query) で返ってくるのは Document オブジェクトのリスト
#（例）：
# [
#     Document(page_content="World War I or the First World War (28 July 1914 – 11 November 1918)..."),
#     Document(page_content="Some wars name themselves. This is the Great War...")
# ]

# To dict
data = {
    "query": question,
    "response": response,
    "context": contexts,
}

what bacteria grow on macconkey agar
who wrote a rose is a rose is a rose


# Evaluation in another LLM

In [20]:
import openai
# # OpenAI クライアントを作成
client = openai.OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# # 使用可能なモデル一覧を取得
# available_models = client.models.list()

# # モデル名を抽出
# model_names = [model.id for model in available_models]

# # すべての利用可能なモデルを表示
# import pprint
# pprint.pprint(model_names)

# # "o3-mini-high" が含まれているか
# print("o3-mini-high" in model_names)

In [21]:
evaluation_prompt = """
You are an expert AI evaluator.
You will evaluate the relevance and accuracy of the given response based on the provided context.

### Evaluation Criteria: ５段階評価
1. **Relevance (1-5):** Does the response directly answer the question?
2. **Accuracy (1-5):** Is the response factually correct given the context?
3. **Justification:** Explain why the response is rated this way.

### Example Format:
Relevance: 4
Accuracy: 5
Justification: The response correctly states the end date of World War I, which matches the context.

### Input:
Query: {query}
Response: {response}
Context: {context}

### Output:
"""

In [22]:
import pandas as pd

# LLM を使って評価を実施する関数
def evaluate_response(record):
    query = record["query"]
    response = record["response"]
    context = "\n".join(record["context"])

    prompt = evaluation_prompt.format(query=query, response=response, context=context)

    completion = client.chat.completions.create(
        model="gpt-4o-mini",  # 必要に応じて変更
        messages=[{"role": "system", "content": "You are an AI assistant evaluating responses."},
                  {"role": "user", "content": prompt}],
        max_tokens=200
    )

    evaluation_text = completion.choices[0].message.content.strip()

    # Relevance と Accuracy のスコアを抽出
    relevance_score = None
    accuracy_score = None
    justification = ""

    for line in evaluation_text.split("\n"):
        if line.startswith("Relevance:"):
            relevance_score = int(line.split(":")[1].strip())
        elif line.startswith("Accuracy:"):
            accuracy_score = int(line.split(":")[1].strip())
        elif line.startswith("Justification:"):
            justification = line.split("Justification:")[1].strip()

    return {
        "Query": query,
        "Response": response,
        "Context": context,
        "Relevance": relevance_score,
        "Accuracy": accuracy_score,
        "Justification": justification
    }

df_dict = pd.DataFrame(data)

# df_dict のデータを LLM で評価
evaluated_results = [evaluate_response(record) for record in df_dict.to_dict('records')]

# DataFrame に変換
df_evaluated = pd.DataFrame(evaluated_results)
df_evaluated

,Query,Response,Context,Relevance,Accuracy,Justification
0,what bacteria grow on macconkey agar,MacConkey agar is designed to selectively isol...,context: ['MacConkey agar is a selective and d...,5,5,The response accurately describes the purpose ...
1,who wrote a rose is a rose is a rose,"Gertrude Stein wrote ""A rose is a rose is a ro...",'Version ridicules the stupidity of court spee...,5,5,The response accurately attributes the phrase ...
